## Implementation of ResNet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def residual_block(x, filters, kernel_size, strides=(1, 1), activation='relu'):
    # Shortcut path
    shortcut = x

    # First convolutional layer
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)

    # Second convolutional layer
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Shortcut connection
    if strides != (1, 1) or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.add([x, shortcut])
    x = layers.Activation(activation)(x)
    return x

def resnet(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # Initial convolutional layer
    x = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Residual blocks
    x = residual_block(x, 64, (3, 3))
    x = residual_block(x, 64, (3, 3))
    x = residual_block(x, 64, (3, 3))

    x = residual_block(x, 128, (3, 3), strides=(2, 2))
    x = residual_block(x, 128, (3, 3))
    x = residual_block(x, 128, (3, 3))

    x = residual_block(x, 256, (3, 3), strides=(2, 2))
    x = residual_block(x, 256, (3, 3))
    x = residual_block(x, 256, (3, 3))

    x = residual_block(x, 512, (3, 3), strides=(2, 2))
    x = residual_block(x, 512, (3, 3))
    x = residual_block(x, 512, (3, 3))

    # Final layers
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create model
    model = tf.keras.Model(inputs, outputs)
    return model

# Example usage:
input_shape = (224, 224, 3)  # Input shape of the images (height, width, channels)
num_classes = 1000  # Number of output classes
model = resnet(input_shape, num_classes)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 64)         9472      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 112, 112, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 112, 112, 64)         0         ['batch_normalization[0][0